<a href="https://colab.research.google.com/github/lalithakre/MovieRecommendationEngine/blob/main/MovieRecommendationEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Movie Recommendation Engine")